In [1]:
# prompt: i want to load dataset from huggingface https://huggingface.co/datasets/AdamCodd/emotion-balanced

!pip install datasets

from datasets import load_dataset

dataset = load_dataset("AdamCodd/emotion-balanced")

dataset


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.7 MB/s eta 0:00:00


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/9.06k [00:00<?, ?B/s]

data%2Ftrain.jsonl:   0%|          | 0.00/1.97M [00:00<?, ?B/s]

data%2Fvalidation.jsonl:   0%|          | 0.00/248k [00:00<?, ?B/s]

data%2Ftest.jsonl:   0%|          | 0.00/244k [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [3]:
!pip install datasets transformers torch pandas numpy scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 74.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 50.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [4]:
import numpy as np
import pandas as pd
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import AutoTokenizer
import torch
from torch.utils.data import Dataset, DataLoader

In [8]:
# Convert to pandas DataFrames for EDA
train_df = pd.DataFrame(dataset['train'])
valid_df = pd.DataFrame(dataset['validation'])
test_df = pd.DataFrame(dataset['test'])

In [9]:
# Define label mapping
label_map = {
    0: 'sadness',
    1: 'joy',
    2: 'love',
    3: 'anger',
    4: 'fear',
    5: 'surprise'
}

In [10]:
# Add text label column for verification
for df in [train_df, valid_df, test_df]:
    df['emotion'] = df['label'].map(label_map)

In [11]:
# Check class distribution
print("Train Class Distribution:")
print(train_df['emotion'].value_counts())
print("\nValidation Class Distribution:")
print(valid_df['emotion'].value_counts())
print("\nTest Class Distribution:")
print(test_df['emotion'].value_counts())

Train Class Distribution:
emotion
anger       2667
love        2667
fear        2667
sadness     2667
joy         2666
surprise    2666
Name: count, dtype: int64

Validation Class Distribution:
emotion
anger       334
surprise    334
fear        333
joy         333
sadness     333
love        333
Name: count, dtype: int64

Test Class Distribution:
emotion
joy         334
fear        334
anger       333
surprise    333
love        333
sadness     333
Name: count, dtype: int64


In [12]:
# Preprocessing for Traditional ML Models (TF-IDF)
def prepare_tfidf_features(train_texts, test_texts):
    vectorizer = TfidfVectorizer(max_features=5000)
    X_train = vectorizer.fit_transform(train_texts)
    X_test = vectorizer.transform(test_texts)
    return X_train, X_test, vectorizer

In [13]:
# Prepare TF-IDF features
X_train_tfidf, X_test_tfidf, tfidf_vectorizer = prepare_tfidf_features(
    train_df['text'], test_df['text']
)

In [14]:
# Save TF-IDF features for later use
np.savez('tfidf_features.npz',
         X_train=X_train_tfidf,
         X_test=X_test_tfidf,
         y_train=train_df['label'],
         y_test=test_df['label'])

In [15]:
# Preprocessing for Deep Learning Models (BERT)
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [16]:
def tokenize_function(examples):
    return tokenizer(
        examples['text'],
        padding='max_length',
        truncation=True,
        max_length=128,
        return_tensors='pt'
    )

In [17]:
# Tokenize all splits
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [18]:
# Set format for PyTorch
tokenized_datasets.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

In [19]:
# Create DataLoaders
batch_size = 32

train_loader = DataLoader(
    tokenized_datasets['train'],
    shuffle=True,
    batch_size=batch_size
)

valid_loader = DataLoader(
    tokenized_datasets['validation'],
    batch_size=batch_size
)

test_loader = DataLoader(
    tokenized_datasets['test'],
    batch_size=batch_size
)


In [20]:
# Custom Dataset Class for future use
class EmotionDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]

        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label)
        }


In [21]:
# Example usage of custom dataset
train_dataset = EmotionDataset(
    train_df['text'].values,
    train_df['label'].values,
    tokenizer,
    max_length=128
)


In [22]:
# Save processed data for later use
torch.save(tokenized_datasets, 'tokenized_datasets.pt')
torch.save(train_loader, 'train_loader.pt')
torch.save(valid_loader, 'valid_loader.pt')
torch.save(test_loader, 'test_loader.pt')

print("Preprocessing completed successfully!")

Preprocessing completed successfully!
